
# Benchmarking GPUs in Kaggle

In this Kaggle notebook there are two examples of benchmarks GPU with pytorch. On one side it uses a model performance and on the other side it is an adaption from my previous CPU benchmark. 

## Benchmark GPU With models
Code from project: https://github.com/ryujaehun/pytorch-gpu-benchmark

Recollecting performance data of Kaggle.  **It must be activated the GPU accelerator option.**

The next block of code executes an extensive benchmark with DataSets from PyTorch on the GPU among differents types (type float, half and double). We are just going to use one type to simplify executions and tests.

In [ ]:
import torch
import torchvision.models as models
import platform
import psutil
import torch.nn as nn
import datetime
import time
import os
import pandas as pd
import argparse
from torch.utils.data import Dataset, DataLoader
import json
torch.backends.cudnn.benchmark = True
# https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
# This flag allows you to enable the inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware. 
# If you check it using the profile tool, the cnn method such as winograd, fft, etc. is used for the first iteration and the best operation is selected for the device.



MODEL_LIST = {

    models.mnasnet:models.mnasnet.__all__[1:],
    models.resnet: models.resnet.__all__[1:],
    models.densenet: models.densenet.__all__[1:],
    models.squeezenet: models.squeezenet.__all__[1:],
    models.vgg: models.vgg.__all__[1:],
    models.mobilenet:models.mobilenet.__all__[1:],
    models.shufflenetv2:models.shufflenetv2.__all__[1:]
}
#ORIGINAL
#precisions=["float","half",'double']
precisions=["float"]

# For post-voltaic architectures, there is a possibility to use tensor-core at half precision.
# Due to the gradient overflow problem, apex is recommended for practical use.
device_name =str(torch.cuda.get_device_name(0))

# Training settings
parser = argparse.ArgumentParser(description='PyTorch Benchmarking')
parser.add_argument('--WARM_UP','-w', type=int,default=5, required=False, help="Num of warm up")
parser.add_argument('--NUM_TEST','-n', type=int,default=50,required=False, help="Num of Test")
parser.add_argument('--BATCH_SIZE','-b', type=int, default=12, required=False, help='Num of batch size')
parser.add_argument('--NUM_CLASSES','-c', type=int, default=1000, required=False, help='Num of class')
parser.add_argument('--NUM_GPU','-g', type=int, default=1, required=False, help='Num of gpus')
parser.add_argument('--folder','-f', type=str, default='result', required=False, help='folder to save results')
args = parser.parse_args()
args.BATCH_SIZE*=args.NUM_GPU
class RandomDataset(Dataset):

    def __init__(self,  length):
        self.len = length
        self.data = torch.randn( 3, 224, 224,length)

    def __getitem__(self, index):
        return self.data[:,:,:,index]

    def __len__(self):
        return self.len

rand_loader = DataLoader(dataset=RandomDataset( args.BATCH_SIZE*(args.WARM_UP + args.NUM_TEST)),
                         batch_size=args.BATCH_SIZE, shuffle=False,num_workers=8)
def train(precision='single'):
    """use fake image for training speed test"""
    target = torch.LongTensor(args.BATCH_SIZE).random_(args.NUM_CLASSES).cuda()
    criterion = nn.CrossEntropyLoss()
    benchmark = {}
    for model_type in MODEL_LIST.keys():
        for model_name in MODEL_LIST[model_type]:
            model = getattr(model_type, model_name)(pretrained=False)
            if args.NUM_GPU > 1:
                model = nn.DataParallel(model,device_ids=range(args.NUM_GPU))
            model=getattr(model,precision)()
            model=model.to('cuda')
            durations = []
            print(f'Benchmarking Training {precision} precision type {model_name} ')
            for step,img in enumerate(rand_loader):
                img=getattr(img,precision)()
                torch.cuda.synchronize()
                start = time.time()
                model.zero_grad()
                prediction = model(img.to('cuda'))
                loss = criterion(prediction, target)
                loss.backward()
                torch.cuda.synchronize()
                end = time.time()
                if step >= args.WARM_UP:
                    durations.append((end - start)*1000)
            print(f'{model_name} model average train time : {sum(durations)/len(durations)}ms')
            del model
            benchmark[model_name] = durations
    return benchmark

def inference(precision='float'):
    benchmark = {}
    with torch.no_grad():
        for model_type in MODEL_LIST.keys():
            for model_name in MODEL_LIST[model_type]:
                model = getattr(model_type, model_name)(pretrained=False)
                if args.NUM_GPU > 1:
                    model = nn.DataParallel(model,device_ids=range(args.NUM_GPU))
                model=getattr(model,precision)()
                model=model.to('cuda')
                model.eval()
                durations = []
                print(f'Benchmarking Inference {precision} precision type {model_name} ')
                for step,img in enumerate(rand_loader):
                    img=getattr(img,precision)()
                    torch.cuda.synchronize()
                    start = time.time()
                    model(img.to('cuda'))
                    torch.cuda.synchronize()
                    end = time.time()
                    if step >= args.WARM_UP:
                        durations.append((end - start)*1000)
                print(f'{model_name} model average inference time : {sum(durations)/len(durations)}ms')
                del model
                benchmark[model_name] = durations
    return benchmark

f"{platform.uname()}\n{psutil.cpu_freq()}\ncpu_count: {psutil.cpu_count()}\nmemory_available: {psutil.virtual_memory().available}"


if __name__ == '__main__':
    folder_name= "../output/results"
    
    device_name=f"{device_name}_{args.NUM_GPU}_gpus_"
    system_configs=f"{platform.uname()}\n\
                     {psutil.cpu_freq()}\n\
                    cpu_count: {psutil.cpu_count()}\n\
                    memory_available: {psutil.virtual_memory().available}"
    gpu_configs=[torch.cuda.device_count(),torch.version.cuda,torch.backends.cudnn.version(),torch.cuda.get_device_name(0)]
    gpu_configs=list(map(str,gpu_configs))
    temp=['Number of GPUs on current device : ','CUDA Version : ','Cudnn Version : ','Device Name : ']

    os.makedirs(folder_name, exist_ok=True)
    with open(os.path.join(folder_name, 'config.json'), 'w') as f:
        json.dump(vars(args), f, indent=2)
    now = datetime.datetime.now()
    
    start_time=now.strftime('%Y/%m/%d %H:%M:%S')
    
    print(f'benchmark start : {start_time}')

    for idx,value in enumerate(zip(temp,gpu_configs)):
        gpu_configs[idx]=''.join(value)
        print(gpu_configs[idx])
    print(system_configs)

    with open(os.path.join(folder_name,"system_info.txt"), "w") as f:
        f.writelines(f'benchmark start : {start_time}\n')
        f.writelines('system_configs\n\n')
        f.writelines(system_configs)
        f.writelines('\ngpu_configs\n\n')
        f.writelines(s + '\n' for s in gpu_configs )

    
    for precision in precisions:
        train_result=train(precision)
        train_result_df = pd.DataFrame(train_result)
        path=f'{folder_name}/{device_name}_{precision}_model_train_benchmark.csv'
        train_result_df.to_csv(path, index=False)

        inference_result=inference(precision)
        inference_result_df = pd.DataFrame(inference_result)
        path=f'{folder_name}/{device_name}_{precision}_model_inference_benchmark.csv'
        inference_result_df.to_csv(path, index=False)

    now = datetime.datetime.now()

    end_time=now.strftime('%Y/%m/%d %H:%M:%S')
    print(f'benchmark end : {end_time}')
    with open(os.path.join(folder_name,"system_info.txt"), "a") as f:
        f.writelines(f'benchmark end : {end_time}\n')

In [ ]:
import pandas as pd
import os,time
import glob
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=True, world_readable=True)
import torchvision.models as models

In [ ]:
MODEL_LIST = {
    'mnasnet':models.mnasnet.__all__[1:],
    'resnet': models.resnet.__all__[1:],
    'densenet': models.densenet.__all__[1:],
    'squeezenet': models.squeezenet.__all__[1:],
    'vgg': models.vgg.__all__[1:],
    'mobilenet':models.mobilenet.__all__[1:],
    'shufflenetv2':models.shufflenetv2.__all__[1:]
}
folder_name='../output/results/'

In [ ]:
print(os.listdir("../output/results"))


In [ ]:
csv_list=glob.glob(folder_name+'/*.csv')
columes=[]
for key,values in MODEL_LIST.items():
    for i in values:
        columes.append((key,i))

In [ ]:
for csv in csv_list:
    df=pd.read_csv(csv)
    df.columns = pd.MultiIndex.from_tuples(columes)
    df.groupby(level=0,axis=1).mean().mean()
#     print(csv)
    title=csv.split('/')[1].split('_benchmark')[0]
    title=title.replace(' ','_')
    df.groupby(level=0,axis=1).mean().mean().iplot(kind='scatter',mode='markers',title=title,yTitle='time(ms)',xTitle='models',asImage=True,filename=title)
    for model in MODEL_LIST.keys():
        df.mean()[model].iplot(kind='scatter',mode='markers',title=model+"_"+title,yTitle='time(ms)',xTitle='models',asImage=True,filename=model+"_"+title)
        time.sleep(1)

## My CPU Benchmark adapted to GPU

Pytorch has hard coded a block size of 512 threads. So there's only one execution per matrix size.

As last benchmark, GPU accelerator has to be activated to use this benchmark.

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Author: Raquel Ricoy

#Benchmark to study Kaggle's GPUs, CPUs and TPUs potential.
#It's going to use Pytorch and to stablish a script to calculate its performance and GFLOPS.

#Install pytorch
#!conda install -y pytorch torchvision -c pytorch

import torch
import platform

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
#print(os.listdir("../input"))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Importing Libraries needed for use torch
import timeit
import torch.utils.benchmark as benchmark
from itertools import product

In [4]:
#Functions obtained from Torch Webpages por PyTorch Benchmarks
def batched_dot_mul_sum(a, b):
    '''Computes batched dot by multiplying and summing'''
    return a.mul(b).sum(-1)


def batched_dot_bmm(a, b):
    '''Computes batched dot by reducing to bmm'''
    a = a.reshape(-1, 1, a.shape[-1])
    b = b.reshape(-1, b.shape[-1], 1)
    return torch.bmm(a, b).flatten(-3)


In [5]:
# Method that do the benchmark and compare results with dot mul sum implementations and vectorSum
def benchMark(sizes,nThreads):
    results = []
    if(len(sizes) == 0):
        print("Parameter 'sizes' has to a have minumun of 1 parameters")
        return
    if(len(nThreads)==0):
        print("Parameter 'nThreads' has to a have minumun of 1 parameters")
    
    for n in sizes:
        # label and sub_label are the rows
        # description is the column
        label = 'Batched dot'
        sub_label = f'[{n}, {n}]'
        x = torch.ones((n, n),device='cuda')
        for num_threads in nThreads:
            results.append(benchmark.Timer(
                stmt='batched_dot_mul_sum(x, x)',
                setup='from __main__ import batched_dot_mul_sum',
                globals={'x': x},
                num_threads=num_threads,
                label=label,
                sub_label=sub_label,
                description='mul/sum',
            ).blocked_autorange(min_run_time=1))
            results.append(benchmark.Timer(
                stmt='batched_dot_bmm(x, x)',
                setup='from __main__ import batched_dot_bmm',
                globals={'x': x},
                num_threads=num_threads,
                label=label,
                sub_label=sub_label,
                description='bmm',
            ).blocked_autorange(min_run_time=1))
    compare = benchmark.Compare(results)
    compare.print()

In [6]:
#The limit dimension of the sizes is [100000,100000]. It is running out of memory with that sizes
sizes = [512,2048,4096,8192,16384,32768]
threads = [1] #We put a single

benchMark(sizes,threads)

AssertionError: Torch not compiled with CUDA enabled